ID #1: 311515381

ID #2: 206337768

# Exercise 2: Decision Trees

In this assignment you will implement a Decision Tree algorithm as learned in class.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
1. Submission includes this notebook only with the exercise number and your ID as the filename. For example: `hw2_123456789_987654321.ipynb` if you submitted in pairs and `hw2_123456789.ipynb` if you submitted the exercise alone.
1. Write **efficient vectorized** code whenever possible. Some calculations in this exercise take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deduction.
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
1. Write your functions in this notebook only. **Do not create Python modules and import them**.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. **Do not import anything else.**

## In this exercise you will perform the following:
1. Practice OOP in python.
2. Implement two impurity measures: Gini and Entropy.
3. Construct a decision tree algorithm.
4. Prune the tree to achieve better results.
5. Visualize your results.

# I have read and understood the instructions: *** YOUR ID HERE ***

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Warmup - OOP in python

Our desicion tree will be implemented using a dedicated python class. Python classes are very similar to classes in Java.


You can use the following [site](https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/) to learn about classes in python.

In [2]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, node):
        self.children.append(node)

In [3]:
n = Node(5)
p = Node(6)
q = Node(7)
n.add_child(p)
n.add_child(q)
n.children

[<__main__.Node at 0x7f9ea05338e0>, <__main__.Node at 0x7f9ea05339a0>]

## Data preprocessing

For the following exercise, we will use a dataset containing mushroom data `agaricus-lepiota.csv`. 

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous
one (=there are only two classes **edible** and **poisonous**). 
    
The dataset contains 8124 observations with 22 features:
1. cap-shape: bell=b,conical=c,convex=x,flat=f,knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
4. bruises: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g,green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

First, we will read and explore the data using pandas and the `.read_csv` method. Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [4]:
# load dataset
data = pd.read_csv('agaricus-lepiota.csv')

One of the advantages of the Decision Tree algorithm is that almost no preprocessing is required. However, finding missing values is always required.

In [5]:
data.dropna(axis=1, inplace=True)

assert data.isna().sum().sum() == 0, 'There are still NA values in the data'

We will split the dataset to `Training` and `Testing` datasets.

In [6]:
from sklearn.model_selection import train_test_split

# Making sure the last column will hold the labels
X, y = data.drop('class', axis=1), data['class']
X = np.column_stack([X,y])
# split dataset using random_state to get the same split each time
X_train, X_test = train_test_split(X, random_state=99)

print("Training dataset shape: ", X_train.shape)
print("Testing dataset shape: ", X_test.shape)

Training dataset shape:  (6093, 22)
Testing dataset shape:  (2031, 22)


In [7]:
y.shape

(8124,)

## Impurity Measures

Impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Implement the functions `calc_gini` and `calc_entropy`. You are encouraged to test your implementation (10 points).

In [8]:
def find_p(data):
    return np.unique(data[:, -1], return_counts=True)[1] / data.shape[0]

In [9]:
def calc_gini(data):
    """
    Calculate gini impurity measure of a dataset.
 
    Input:
    - data: any dataset where the last column holds the labels.
 
    Returns the gini impurity.    
    """
    
    p = find_p(data)
    gini = 1 - (p**2).sum()

    return gini

In [10]:
def calc_entropy(data):
    """
    Calculate the entropy of a dataset.

    Input:
    - data: any dataset where the last column holds the labels.

    Returns the entropy of the dataset.    
    """    
    p = find_p(data)

    entropy = - (p * np.log2(p)).sum()

    return entropy

In [11]:
##### Your Tests Here #####
calc_gini(X), calc_entropy(X)

(0.4995636322379775, 0.9993703627906085)

## Goodness of Split

Given a feature the Goodnees of Split measures the reduction in the impurity if we split the data according to the feature.
$$
\Delta\varphi(S, A) = \varphi(S) - \sum_{v\in Values(A)} \frac{|S_v|}{|S|}\varphi(S_v)
$$

NOTE: you can add more parameters to the function and you can also add more returning variables (The given parameters and the given returning variable should not be touch). (10 Points)

In [12]:
def goodness_of_split(data, feature, impurity_func, gain_ratio=False):
    """
    Calculate the goodness of split of a dataset given a feature and impurity function.

    Input:
    - data: any dataset where the last column holds the labels.
    - feature: the feature index.
    - impurity func: a function that calculates the impurity.
    - gain_ratio: goodness of split or gain ratio flag.

    Returns the goodness of split.
    """
    N = data.shape[0]
    impurity = impurity_func(data)
    gain = 0

    feature_values = np.unique(data[:, feature])

    for value in feature_values:
        value_data = data[data[:, feature] == value]
        n = value_data.shape[0]
        gain += ((n / N) * impurity_func(value_data))

    goodness = impurity - gain

    if gain_ratio:
        probs = np.unique(data[:, feature], return_counts=True)[1] / data.shape[0]
        split_information = -(np.log2(probs) * probs).sum()
        goodness = impurity / split_information

    return goodness

## Building a Decision Tree

Use a Python class to construct the decision tree. Your class should support the following functionality:

1. Initiating a node for a decision tree. You will need to use several class methods and class attributes and you are free to use them as you see fit. We recommend that every node will hold the feature and value used for the split and its children.
2. Your code should support both Gini and Entropy as impurity measures. 
3. The provided data includes categorical data. In this exercise, when splitting a node create the number of children needed according to the attribute unique values.

Complete the class `DecisionNode`. The structure of this class is entirely up to you. 

Complete the function `build_tree`. This function should get the training dataset and the impurity as inputs, initiate a root for the decision tree and construct the tree according to the procedure you learned in class. (30 points)

In [13]:
max_unique = 0
col_name = None
for col in data.columns.tolist():
    if np.unique(data[col]).size > max_unique:
        max_unique = np.unique(data[col]).size
        col_name = col
print(f"There are at most {max_unique} unique values in a feature in the data set\nThis number is achieved in {col_name}\nSo the maximal value of the degrees of freedom is {max_unique - 1} ")

There are at most 12 unique values in a feature in the data set
This number is achieved in gill-color
So the maximal value of the degrees of freedom is 11 


In [14]:
chi_square_table_dict = {1: 0.00393, 2: 0.103, 3: 0.352, 4: 0.711, 5: 1.145, 6: 1.635, 7: 2.833, 8: 3.49, 9: 3.325, 10: 3.940, 11: 5.578}

In [15]:
class DecisionNode:
    """
    This class will hold everything you require to construct a decision tree.
    The structure of this class is up to you. However, you need to support basic
    functionality as described above. It is highly recommended that you
    first read and understand the entire exercise before diving into this class.
    """

    def __init__(self, feature, value, depth, data, num_samples=0, feature_name=None, parent_name=None):
        self.feature = feature  # column index of criteria being tested
        self.value = value
        self.depth = depth
        self.data = data
        self.children = []
        self.chi_squared = 0
        self.pred = None
        self.num_samples = num_samples
        self.feature_name = feature_name
        self.parent_name = parent_name

    def add_child(self, node):
        self.children.append(node)

    def chi_square_test(self):
        feature_values = np.unique(self.data[:, self.feature])
        temp_df = self.data.iloc[:, [self.feature, -1]]
        P_e = ((temp_df[:, -1] == 'e').sum()) / temp_df.shape[0]
        P_p = ((temp_df[:, -1] == 'p').sum()) / temp_df.shape[0]
        chi = 0
        for value in feature_values:
            feature_col = temp_df[:, 0]
            target_col = temp_df[:, -1]
            Df = (feature_col == value).sum()
            Pf = ((feature_col == value) & (target_col == 'e')).sum()
            Nf = ((feature_col == value) & (target_col == 'p')).sum()
            E0 = Df * P_e
            E1 = Df * P_p
            chi += (((Pf - E0) ** 2) / E0) + (((Nf - E1) ** 2) / E1)
        self.chi_squared = chi


In [16]:
def build_tree(data, impurity, gain_ratio=False, min_samples_split=1, max_depth=1000, chi=1, max_goodness=0, max_goodness_index=0):
    """
    Build a tree using the given impurity measure and training dataset.
    You are required to fully grow the tree until all leaves are pure.

    Input:
    - data: the training dataset.
    - impurity: the chosen impurity measure. Notice that you can send a function
                as an argument in python.
    - gain_ratio: goodness of split or gain ratio flag
    - min_samples_split: the minimum number of samples required to split an internal node
    - max_depth: the allowable depth of the tree

    Output: the root node of the tree.
    """
    root = None

    df = pd.DataFrame(data, columns=[f"X{i}" for i in range(data.shape[1])])

    queue = []

    for idx in range(df.iloc[:, :-1].shape[1]):

        goodness = goodness_of_split(data, idx, impurity, gain_ratio)

        if goodness > max_goodness:
            max_goodness = goodness
            max_goodness_idx = idx

    feature_labels = np.unique(df.iloc[:, max_goodness_idx]).tolist()

    root = DecisionNode(None, None, 0, df, len(df), feature_name='root')

    for label in feature_labels:
        node = DecisionNode(max_goodness_idx, label, 1, df[df.iloc[:, max_goodness_idx] == label],
                            feature_name=f"X{max_goodness_idx}", parent_name=root.feature_name)
        root.add_child(node)
        queue.insert(0, node)

    if np.isclose(impurity(data), 0):
        return root

    tree_depth = 0

    while len(queue) > 0:

        cur_node = queue.pop()

        print(f"{'-' * 10}\tprocessing '{cur_node.feature_name}' == '{cur_node.value}' --> "
              f"child of {cur_node.parent_name}\t{'-' * 10}")

        cur_node.num_samples = cur_node.data.shape[0]
        tree_depth = max(cur_node.depth, tree_depth)

        labels, counts = np.unique(cur_node.data.iloc[:, -1], return_counts=True)
        cur_node.pred = labels[np.argmax(counts)]

        if cur_node.depth >= max_depth:
            continue

        if cur_node.num_samples <= min_samples_split:
            continue

        if np.isclose(impurity(cur_node.data.to_numpy()), 0):
            continue

        dof = np.unique(df.iloc[:, cur_node.feature]).size - 1

        if (chi < 1) and dof > 0:
            cur_node.chi_square_test()
            if chi_square_table_dict[dof] > cur_node.chi_squared:
                continue

        cur_node.data = cur_node.data[cur_node.data.iloc[:, cur_node.feature] == cur_node.value]
        cur_node.data = cur_node.data.drop([cur_node.feature_name], axis=1)

        max_goodness = 0
        max_goodness_idx = 0
        temp_data = cur_node.data.to_numpy()

        for idx in range(cur_node.data.iloc[:, :-1].shape[1]):

            goodness = goodness_of_split(temp_data, idx, impurity, gain_ratio)

            if goodness > max_goodness:
                max_goodness = goodness
                max_goodness_idx = idx

        feature_labels = np.unique(cur_node.data.iloc[:, max_goodness_idx]).tolist()

        node_depth = cur_node.depth + 1

        for label in feature_labels:
            temp_node = DecisionNode(max_goodness_idx, label, node_depth, cur_node.data,
                                     feature_name=cur_node.data.columns[max_goodness_idx],
                                     parent_name=f"{cur_node.feature_name} = {cur_node.value}")
            cur_node.add_child(temp_node)
            queue.insert(0, temp_node)


    return root

In [17]:
# python supports passing a function as an argument to another function.
tree_gini = build_tree(data=X_train, impurity=calc_gini) # gini and goodness of split
tree_entropy = build_tree(data=X_train, impurity=calc_entropy) # entropy and goodness of split
tree_entropy_gain_ratio = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True) # entropy and gain ratio

----------	processing 'X4' == 'a' --> child of root	----------
----------	processing 'X4' == 'c' --> child of root	----------
----------	processing 'X4' == 'f' --> child of root	----------
----------	processing 'X4' == 'l' --> child of root	----------
----------	processing 'X4' == 'm' --> child of root	----------
----------	processing 'X4' == 'n' --> child of root	----------
----------	processing 'X4' == 'p' --> child of root	----------
----------	processing 'X4' == 's' --> child of root	----------
----------	processing 'X4' == 'y' --> child of root	----------
----------	processing 'X2' == 'n' --> child of X4 = a	----------
----------	processing 'X2' == 'w' --> child of X4 = a	----------
----------	processing 'X2' == 'y' --> child of X4 = a	----------
----------	processing 'X6' == 'c' --> child of X4 = c	----------
----------	processing 'X6' == 'w' --> child of X4 = c	----------
----------	processing 'X10' == 'f' --> child of X4 = f	----------
----------	processing 'X10' == 'k' --> chi

## Tree evaluation

Complete the functions `predict` and `calc_accuracy`. (10 points)

In [52]:
def predict(node, instance):
    """
    Predict a given instance using the decision tree
 
    Input:
    - root: the root of the decision tree.
    - instance: an row vector from the dataset. Note that the last element 
                of this vector is the label of the instance.
 
    Output: the prediction of the instance.
    """
    pred = None
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    while len(node.children) > 0:

        best_feature = node.children[0].feature
        instance_value = instance[best_feature]

        for child in node.children:

            if child.value == instance_value:

                node = child
        
        assert node != None, 'instance does not exist in dataset'
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return node.pred

In [53]:
def calc_accuracy(node, dataset):
    """
    Predict a given dataset using the decision tree
 
    Input:
    - node: a node in the decision tree.
    - dataset: the dataset on which the accuracy is evaluated
 
    Output: the accuracy of the decision tree on the given dataset (%).
    """
    accuracy = 0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    target_values = dataset[:, -1]
    predictions = np.array([predict(node, instance) for instance in dataset])
    accuracy = (target_values == predictions).sum() / dataset.shape[0] * 100
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return accuracy 

After building the three trees using the training set, you should calculate the accuracy on the test set. For each tree print the training and test accuracy. Select the tree that gave you the best test accuracy. For the rest of the exercise, use that tree (when you asked to build another tree use the same impurity function and same gain_ratio flag). 

In [55]:
#### Your code here ####
tree_gini_accuracy = round(calc_accuracy(tree_gini,X_test),2)
tree_entropy_accuracy = round(calc_accuracy(tree_entropy,X_test),2)
tree_entropy_gain_ratio_accuracy = round(calc_accuracy(tree_entropy_gain_ratio,X_test),2)

print(f"The accuracy for the gini tree is {tree_gini_accuracy}%")
print(f"The accuracy for the entropy tree is {tree_entropy_accuracy}%")
print(f"The accuracy for the entropy with the gain ratio tree is {tree_entropy_gain_ratio_accuracy}%")

## Depth pruning

(15 points)

Consider the following max_depth values: [1, 2, 3, 4, 5, 6, 7, 8]. For each value, construct a tree and prune it according to the max_depth value = don't let the tree to grow beyond this depth. Next, calculate the training and testing accuracy.<br>
On a single plot, draw the training and testing accuracy as a function of the max_depth. Mark the best result on the graph with red circle.

In [1]:
#### Your code here ####

## Min Samples Split

(15 points)

Consider the following min_samples_split values: [1, 5, 10, 20, 50]. For each value, construct a tree and prune it according to the min_samples_split value = don't split a node if the number of sample in it is less or equal to the min_samples_split value. Next, calculate the training and testing accuracy.<br>
On a single plot, draw the training and testing accuracy as a function of the min_samples_split. Mark the best result on the graph with red circle. (make sure that the x-axis ticks represent the values of min_samples_split)

In [ ]:
#### Your code here ####

Build the best 2 trees:
1. tree_max_depth - the best tree according to max_depth pruning
1. tree_min_samples_split - the best tree according to min_samples_split pruning

In [ ]:
#### Your code here ####

## Number of Nodes

(5 points)

Complete the function counts_nodes and print the number of nodes in each tree and print the number of nodes of the two trees above

In [ ]:
def count_nodes(node):
    """
    Count the number of node in a given tree
 
    Input:
    - node: a node in the decision tree.
 
    Output: the number of node in the tree.
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    pass
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    

## Print the tree

Complete the function `print_tree`. Your tree should be visualized clearly. You can use the following example as a reference:
```
[ROOT, feature=X0],
  [X0=a, feature=X2]
    [X2=c, leaf]: [{1.0: 10}]
    [X2=d, leaf]: [{0.0: 10}]
  [X0=y, feature=X5], 
    [X5=a, leaf]: [{1.0: 5}]
    [X5=s, leaf]: [{0.0: 10}]
  [X0=e, leaf]: [{0.0: 25, 1.0: 50}]
```
In each brackets:
* The first argument is the parent feature with the value that led to current node
* The second argument is the selected feature of the current node
* If the current node is a leaf, you need to print also the labels and their counts

(5 points)

In [ ]:
# you can change the function signeture
def print_tree(node, depth=0, parent_feature='ROOT', feature_val='ROOT'):
    '''
    prints the tree according to the example above

    Input:
    - node: a node in the decision tree

    This function has no return value
    '''
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    pass
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

print the tree with the best test accuracy and with less than 50 nodes (from the two pruning methods)

In [ ]:
#### Your code here ####
